<a href="https://colab.research.google.com/github/minhthy1016/Credit-Innovators/blob/minhthy1016-patch-1/webpush_backfill_2020_08_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Main script

In [1]:
import logging
import datetime
import os
import time
import requests
import json
import pandas as pd
import pandas_gbq
import google.auth
import google.auth.transport.requests
from google.cloud import bigquery
from datetime import date, timedelta
from json import dumps

import itertools
from google.oauth2 import service_account
from google.auth import compute_engine
from google.cloud import storage
import os,errno 

###SERVICE ACCOUNT
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'/content/hhg-cdm-bcdeee60c0e1.json'
def query_bq():
    client = bigquery.Client.from_service_account_json(r'/content/hhg-cdm-bcdeee60c0e1.json')
    query_text="""
SELECT
  partner_id,
  campaign_id,
  api_key
FROM
  `sandbox-247611.hevo.th_wb`
"""
    print(query_text)
    query_job = client.query(query_text)
    results = query_job.result().to_dataframe()
    return results
campaign_list = query_bq()
def get_data(partner_id, start_date, end_date, campaign_id, api_key):
    url = "https://web-push.apis.useinsider.com/v1/statistics/campaign"
    payload = {"partner_id": partner_id, "start_date": start_date, "end_date": end_date, "campaign_id": campaign_id}
    #    print(payload)
    headers = {
        'content-type': "application/json",
        'authorization': "Bearer {}".format(api_key)
    }
    response = requests.request("PUT", url, data=json.dumps(payload), headers=headers)
    if response.status_code==200:
      stat = response.json().get('statistics')
      print(stat)
      if stat:
        stat["campaign_id"] = campaign_id
        stat["partner_id"] = partner_id
        return pd.DataFrame(stat)
      else:
        return pd.DataFrame()
    else:
      return pd.DataFrame()
#   all_df.to_gbq('hhg_lead_mailchimp.hhg_push_stats', project_id='hhg-ads', if_exists='replace', credentials=credentials)
def main():
    global date
    credentials = service_account.Credentials.from_service_account_file(
        r'/content/hhg-cdm-bcdeee60c0e1.json')
    # file_path='/home/airflow/gcs/dags/new_mailchimp_test/webpush_backup.json'
    # with open(file_path, 'w') as filetowrite:
    #    filetowrite.write(json.dumps(all_df))
    global all_df
    all_df = pd.DataFrame()
    start_date = date(2021, 6, 11)
    end_date = date(2021, 6, 11)
    delta = end_date - start_date
    for i in range(delta.days +1):
        day = start_date + timedelta(days=i)
        print(day)
        #print(datetime.date.strftime(r'%Y-%m-%d'))
   # date_to_load = (date.today() - timedelta(days=1)).strftime(r'%Y-%m-%d')  # days=1 means yesterday
    start_date=start_date.strftime("%Y-%m-%d")
    end_date=end_date.strftime("%Y-%m-%d")
    for index, row in campaign_list.iterrows():           # print 5 results
        data = get_data(str(row['partner_id']), start_date, end_date, str(row['campaign_id']),
                        str(row['api_key']))
        if data.shape[0]>0:
            data['start_date']=start_date            # get 'date' column
            data['end_date']=end_date
            all_df=all_df.append(data)
            
        time.sleep(2)
    
    # print the result
    print(all_df)
    # get 'device' column from 'deviceLogs'
    #all_df['date'] = sdate
    all_df['devices']=all_df['deviceLogs'].apply(lambda x: x.get('device'))
    #save to csv
    all_df.to_csv('backfill_20210611_th.csv', index=False, encoding='utf8')
    
    
main()


SELECT
  partner_id,
  campaign_id,
  api_key
FROM
  `sandbox-247611.hevo.th_wb`

2021-06-11
{'name': "Goat's Milk", 'type': 'bulk', 'status': 'Active', 'isRecurring': False, 'sent': '0', 'clicks': '0', 'ctr': '0.00%', 'conversions': 0, 'conversionRate': '0.00%', 'deviceLogs': [{'device': 'Desktop', 'sent': '0', 'clicks': '0', 'ctr': '0.00%'}, {'device': 'Mobile', 'sent': '0', 'clicks': '0', 'ctr': '0.00%'}]}
{'name': 'Bad lifestyles', 'type': 'bulk', 'status': 'Active', 'isRecurring': False, 'sent': '0', 'clicks': '0', 'ctr': '0.00%', 'conversions': 0, 'conversionRate': '0.00%', 'deviceLogs': [{'device': 'Desktop', 'sent': '0', 'clicks': '0', 'ctr': '0.00%'}, {'device': 'Mobile', 'sent': '0', 'clicks': '0', 'ctr': '0.00%'}]}
{'name': 'Swallowing semen', 'type': 'trigger', 'status': 'Passive', 'isRecurring': False, 'sent': '0', 'clicks': '0', 'ctr': '0.00%', 'conversions': 0, 'conversionRate': '0.00%', 'deviceLogs': [{'device': 'Desktop', 'sent': '0', 'clicks': '0', 'ctr': '0.00%'}, {

In [ ]:
all_df

,name,type,status,isRecurring,sent,clicks,ctr,conversions,conversionRate,deviceLogs,campaign_id,partner_id,start_date,end_date,devices
0,Exit Intent Article Page,trigger,Active,False,33,0,0.00%,0,0.00%,"{'device': 'Desktop', 'sent': '21', 'clicks': ...",4,10004526,2021-01-01,2021-01-01,Desktop
1,Exit Intent Article Page,trigger,Active,False,33,0,0.00%,0,0.00%,"{'device': 'Mobile', 'sent': '12', 'clicks': '...",4,10004526,2021-01-01,2021-01-01,Mobile


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Test Gsheet - Airflow **

In [ ]:
import logging
import datetime
import os
import time
import requests
import json
import pandas as pd
import pandas_gbq
import google.auth
import google.auth.transport.requests
from google.cloud import bigquery
from datetime import date, timedelta
from json import dumps

import itertools
from google.oauth2 import service_account
from google.auth import compute_engine
from google.cloud import storage
import os,errno 

###SERVICE ACCOUNT
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'/content/hhg-ads-4e51d07bf77e.json'
def query_bq():
    client = bigquery.Client.from_service_account_json(r'/content/hhg-ads-4e51d07bf77e.json')
    query_text="""
SELECT Date, Landing_page, Search_query, Device, Impressions, Clicks, CTR, Average_position FROM `hhg-ads.python_training.gsc_test`
"""
    print(query_text)
    query_job = client.query(query_text)
    results = query_job.result().to_dataframe()
    return results
campaign_list = query_bq()
def get_data(Date, Landing_page, Search_query, Device, Impressions, Clicks, CTR, Average_position):
    url = "https://docs.google.com/spreadsheets/d/1cZhd3i3QfPUxHlJrT2xan92bybB84QOGZuNkloxT_yw/edit#gid=0"
    payload = {"Date": Date, "Landing_page": Landing_page, "Search_query": Search_query, "Device": Device, "Impressions": Impressions, 
               "Clicks": Clicks,"CTR": CTR, "Average_position": Average_position}
    #    print(payload)
    headers = {
        'content-type': "application/json",
        'authorization': "Bearer {}".format(api_key)
    }
    response = requests.request("PUT", url, data=json.dumps(payload), headers=headers)
    if response.status_code==200:
      stat = response.json().get('statistics')
      print(stat)
      if stat:
        stat["Landing_page"] = Landing_page
        stat["Search_query"] = Search_query
        stat["Device"] = Device
        stat["Impressions"] = Impressions
        stat["Clicks"] = Clicks
        stat["CTR"] = CTR
        stat["Average_position"] = Average_position
        return pd.DataFrame(stat)
      else:
        return pd.DataFrame()
    else:
      return pd.DataFrame()
#   all_df.to_gbq('hhg_lead_mailchimp.hhg_push_stats', project_id='hhg-ads', if_exists='replace', credentials=credentials)
def main():
    global date
    credentials = service_account.Credentials.from_service_account_file(
        r'/content/hhg-ads-4e51d07bf77e.json')
    # file_path='/home/airflow/gcs/dags/new_mailchimp_test/webpush_backup.json'
    # with open(file_path, 'w') as filetowrite:
    #    filetowrite.write(json.dumps(all_df))
    global all_df
    all_df = pd.DataFrame()
    date = date(2020, 9, 26)
    
        #print(datetime.date.strftime(r'%Y-%m-%d'))
   # date_to_load = (date.today() - timedelta(days=1)).strftime(r'%Y-%m-%d')  # days=1 means yesterday
    date=start_date.strftime("%Y-%m-%d")
    #payload = {"Date": Date, "Landing_page": Landing_page, "Search_query": Search_query, "Device": Device, "Impressions": Impressions, 
              # "Clicks": Clicks,"CTR": CTR, "Average_position": Average_position}
    for index, row in campaign_list.iterrows():           # print 5 results
        data = get_data(date, str(row['Landing_page']), str(row['Search_query']), str(row['Device']), int(row['Impressions']), int(row['Clicks']),
                        float(row['CTR']), float(row['Average_position']))
        if data.shape[0]>0:
            data['date']=date            # get 'date' column
            #data['end_date']=end_date
            all_df=all_df.append(data)
            
        time.sleep(2)
    
    # print the result
    print(all_df)
    # get 'device' column from 'deviceLogs'
    #all_df['date'] = sdate
    #all_df['devices']=all_df['deviceLogs'].apply(lambda x: x.get('device'))
    #save to csv
    all_df.to_csv('testgsheet.csv', index=False, encoding='utf8')
    
    
main()


SELECT Date, Landing_page, Search_query, Device, Impressions, Clicks, CTR, Average_position FROM `hhg-ads.python_training.gsc_test`



BadRequest: ignored

## #TEST BACKFILL SEPTEMBER

In [ ]:
import logging
import datetime
import os
import time
import requests
import json
import pandas as pd
import pandas_gbq
import google.auth
import google.auth.transport.requests
from google.cloud import bigquery
from datetime import date, timedelta
from json import dumps

import itertools
from google.oauth2 import service_account
from google.auth import compute_engine
from google.cloud import storage
import os,errno 

###SERVICE ACCOUNT
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'/content/hhg-cdm-bcdeee60c0e1.json'
def query_bq():
    client = bigquery.Client.from_service_account_json(r'/content/hhg-cdm-bcdeee60c0e1.json')
    query_text="""
SELECT
  partner_id,
  campaign_id,
  api_key
FROM
  `hhg-ods.hhg_web_push.hhg_web_push_insider_statistics_credential`
"""
    print(query_text)
    query_job = client.query(query_text)
    results = query_job.result().to_dataframe()
    return results
campaign_list = query_bq()
def get_data(partner_id, start_date, end_date, campaign_id, api_key):
    url = "https://web-push.apis.useinsider.com/v1/statistics/campaign"
    payload = {"partner_id": partner_id, "start_date": start_date, "end_date": end_date, "campaign_id": campaign_id}
    #    print(payload)
    headers = {
        'content-type': "application/json",
        'authorization': "Bearer {}".format(api_key)
    }
    response = requests.request("PUT", url, data=json.dumps(payload), headers=headers)
    if response.status_code==200:
      stat = response.json().get('statistics')
      print(stat)
      if stat:
        stat["campaign_id"] = campaign_id
        stat["partner_id"] = partner_id
        return pd.DataFrame(stat)
      else:
        return pd.DataFrame()
    else:
      return pd.DataFrame()
#   all_df.to_gbq('hhg_lead_mailchimp.hhg_push_stats', project_id='hhg-ads', if_exists='replace', credentials=credentials)
def main():
    global date
    credentials = service_account.Credentials.from_service_account_file(
        r'/content/hhg-cdm-bcdeee60c0e1.json')
    # file_path='/home/airflow/gcs/dags/new_mailchimp_test/webpush_backup.json'
    # with open(file_path, 'w') as filetowrite:
    #    filetowrite.write(json.dumps(all_df))
    global all_df
    all_df = pd.DataFrame()
    start_date = date(2020, 9, 18)
    end_date = date(2020, 9, 18)
    delta = end_date - start_date
    for i in range(delta.days +1):
        day = start_date + timedelta(days=i)
        print(day)
        #print(datetime.date.strftime(r'%Y-%m-%d'))
   # date_to_load = (date.today() - timedelta(days=1)).strftime(r'%Y-%m-%d')  # days=1 means yesterday
    start_date=start_date.strftime("%Y-%m-%d")
    end_date=end_date.strftime("%Y-%m-%d")
    for index, row in campaign_list.iterrows():           # print 5 results
        data = get_data(str(row['partner_id']), start_date, end_date, str(row['campaign_id']),
                        str(row['api_key']))
        if data.shape[0]>0:
            data['start_date']=start_date            # get 'date' column
            data['end_date']=end_date
            all_df=all_df.append(data)
            
        time.sleep(2)
    
    # print the result
    print(all_df)
    # get 'device' column from 'deviceLogs'
    #all_df['date'] = sdate
    all_df['devices']=all_df['deviceLogs'].apply(lambda x: x.get('device'))
    #save to csv
    all_df.to_csv('results.csv', index=False, encoding='utf8')
    
    
main()


SELECT
  partner_id,
  campaign_id,
  api_key
FROM
  `hhg-ods.hhg_web_push.hhg_web_push_insider_statistics_credential`

2020-09-18
{'name': 'trigger', 'type': 'trigger', 'status': 'Passive', 'isRecurring': False, 'sent': '0', 'clicks': '0', 'ctr': '0.00%', 'conversions': '0', 'conversionRate': '0.00%', 'deviceLogs': [{'device': 'Desktop', 'sent': '0', 'clicks': '0', 'ctr': '0.00%'}, {'device': 'Mobile', 'sent': '0', 'clicks': '0', 'ctr': '0.00%'}]}
{'name': 'trigger', 'type': 'trigger', 'status': 'Passive', 'isRecurring': False, 'sent': '0', 'clicks': '0', 'ctr': '0.00%', 'conversions': '0', 'conversionRate': '0.00%', 'deviceLogs': [{'device': 'Desktop', 'sent': '0', 'clicks': '0', 'ctr': '0.00%'}, {'device': 'Mobile', 'sent': '0', 'clicks': '0', 'ctr': '0.00%'}]}
{'name': 'trigger', 'type': 'trigger', 'status': 'Passive', 'isRecurring': False, 'sent': '0', 'clicks': '0', 'ctr': '0.00%', 'conversions': '0', 'conversionRate': '0.00%', 'deviceLogs': [{'device': 'Desktop', 'sent': '0', '